In [1]:
import os
import sys 
import pandas as pd 
import seaborn as sns


KeyboardInterrupt



In [ ]:
sys.path.append("../")
sys.path.append("../../")

In [ ]:
from src.utils.TableConverter import md

class Falcon():

    def __init__(self, dir_path) -> None:
        self.dir_path = dir_path

In [ ]:
path = "/scratch/work/koutchc1/datasets/falconcode/falconcode_v1_table_problems.csv"
df = pd.read_csv(path)
df

In [ ]:
def load_subset_with_missing_data(df):
    # Remove duplicate ids for project (same content, multiple semesters)
    df = df.drop_duplicates("id")
    df = df.fillna("")
    df = df.sort_values(by="id")
    # We do not have access to the external ressources (all the projects full description)
    # for assignments of type project, and they will be graded manually by instructors anyway 
    df = df[df.type != "project"]
    # So far, we do not care about the skills for annotation
    df = df.loc[:, :'max_score']
    # Format the prompt for readability
    df["prompt"] = df["prompt"].apply(md)
    df = df[["You have been provided with" in prompt for prompt in df["prompt"]]]
    df = df.reset_index(drop=True)
    return df

sub_df = load_subset_with_missing_data(df)

In [ ]:
print(df[[".csv" in testcase for testcase in df["testcase"]]].prompt.iloc[0])

In [ ]:
# Annotation loop
from tqdm.notebook import tqdm

def annotate(missing_data_df):
    missing = []
    for i, row in tqdm(missing_data_df.iterrows()):
        
        print("**TESTING SCRIPT**")
        print(row["testcase"])
        print()
        print(row["prompt"])
        print()
        
        missing_files = input("Missing files (including startercode)")
        missing.append({"files": missing_files})
        print("------------")
        print()
            
    missing = pd.DataFrame(missing)
    print(missing)
    
    return pd.concat([missing_data_df, missing], axis=1)

In [ ]:
def extract_what_was_provided(statement, n_words=5):
    start = statement.find("You have been provided with")
    provided = statement[start:].split(" ")[5:]
    if provided[0] == 'a': provided = provided[1:] 
    provided = provided[:n_words]
    provided = " ".join(provided)
    return provided

In [ ]:
provided.unique()

In [ ]:
def map_provided_to_category(subdf):
    subdf["extract"] = sub_df.prompt.apply(extract_what_was_provided)
    categories = ["function", "program", "file", "data_structure"]
    missing = []
    for provided, testcase in subdf[["extract", "testcase"]].to_numpy():
        file = ""
        words = testcase.split(" ")
        file = list(set([w for w in words if '.csv' in w or '.txt' in w]))
        if not file:
            words = provided.split(" ")
            file = list(set([w for w in words if '.csv' in w or '.txt' in w]))
        
        file = ",".join(file) if file else "startercode"
        information = {"name": file}
        missing.append(information)
        
    return pd.concat([subdf, pd.DataFrame(missing)], axis=1)

In [ ]:
missing_data_df = map_provided_to_category(sub_df)
missing_data_df

In [ ]:
missing_data_df = missing_data_df[["id", "course_id", "type", "extract", "name"]]
missing_data_df

In [ ]:
missing_data_df.to_csv("problems_df_missing_information.csv")

In [ ]:
missing_data_df.name.unique()

### Analysis of which tables are useful

In [ ]:
print(df.loc[df.problem_id == "Airstrike"]["prompt"])
print(df.loc[df.problem_id == "Airstrike"]["testcase"])

In [ ]:
path = "/scratch/work/koutchc1/datasets/falconcode/falconcode_v1_merged.csv"
runs_df = pd.read_csv(path, index_col=0)
runs_df

In [ ]:
print(runs_df.loc[df.problem_id == "Airstrike"]["source_code"].iloc[0])